In [1]:
%load_ext autoreload
%autoreload
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from models.ResModel import ResModel
from models.Model import Model
from models.JointModel import JointModel
from models.MonoSF import MonoSceneFlow
from losses import Loss
from datasets.kitti_2015_train import KITTI_2015_MonoSceneFlow
from augmentations import Augmentation_Resize_Only, Augmentation_SceneFlow
from collections import OrderedDict
from datasets.kitti_raw_monosf import KITTI_Odom_Test
import numpy as np

class Args:
    cuda = True
    use_bn = False
    momentum = 0.9
    beta = 0.999
    weight_decay=0.0
    train_exp_mask=False
    train_census_mask=False
    model_name = "joint"
    encoder_name='pwc'
    disp_pts_w = 0.0
    flow_pts_w = 0.2
    sf_sm_w = 200
    disp_sm_w = 0.2
    do_pose_c2f = False
    ssim_w = 0.85
    disp_smooth_w = 0.1
    mask_reg_w = 0.0
    num_examples = 200
    static_cons_w = 0.0
    mask_cons_w = 0.0
    mask_sm_w = 0.0
    batch_size = 1
    flow_diff_thresh = 0.05
    pt_encoder = False
    num_scales = 4
    evaluation = True
    use_disp_min = False
    flow_reduce_mode = "sum"
    apply_flow_mask = False
    apply_mask = True 
    mask_thresh = 0.5
    use_bottleneck = False
    use_pose_corr = True
    flow_sm_w = 200
    flow_cycle_w = 0.0
    use_static_mask = False
    disp_lr_w = 0.1

args = Args()

model = JointModel(args).cuda()

loss = Loss(args).cuda()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"The model has {num_params} learnable parameters")

state_dict = torch.load('pretrained/pose_corr.ckpt')['model']
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model = model.eval()

dataset_09 = KITTI_Odom_Test(args, "/external/datasets/kitti_data_jpg/", "09")
dataloader_09 = DataLoader(dataset_09, shuffle=False, batch_size=1, pin_memory=True)
dataset_10 = KITTI_Odom_Test(args, "/external/datasets/kitti_data_jpg/", "10")
dataloader_10 = DataLoader(dataset_10, shuffle=False, batch_size=1, pin_memory=True)

aug = Augmentation_Resize_Only(args).cuda()

The model has 9560098 learnable parameters


In [2]:
# from https://github.com/tinghuiz/SfMLearner
def compute_ate(gtruth_xyz, pred_xyz):
    alignment_error = pred_xyz - gtruth_xyz
    rmse = np.sqrt(sum(alignment_error ** 2)) / gtruth_xyz.shape[0]
    return rmse

seq_09_ates = []
seq_09_res = []
for i, data in enumerate(tqdm(dataloader_09)):
    with torch.no_grad():
        # Get input and target tensor keys
        input_keys = list(filter(lambda x: "input" in x, data.keys()))
        target_keys = list(filter(lambda x: "target" in x, data.keys()))
        tensor_keys = input_keys + target_keys
        
        # Possibly transfer to Cuda
        for k, v in data.items():
            if k in tensor_keys:
                data[k] = v.cuda(non_blocking=True)
                
        aug_data = aug(data)
        out = model(aug_data)
        pose_t = out['pose_b'][0][:, :3, 3].double().cpu()
        gt_t = data['target_pose'][:, :3, 3].double().cpu()
        
        pose_R = out['pose_b'][0][:, :3, :3].double().cpu()[0]
        gt_R = data['target_pose'][:, :3, :3].double().cpu()[0]
        
        R = torch.matmul(gt_R, torch.inverse(pose_R))
        s = np.linalg.norm([R[0, 1]-R[1, 0],
                            R[1, 2]-R[2, 1],
                            R[0, 2]-R[2, 0]])
        c = np.trace(R) - 1
        RE = np.arctan2(s, c)
        
        ate = compute_ate(gt_t, pose_t)
        seq_09_ates.append(np.array(ate))
        seq_09_res.append(RE)
        
seq_09_ates = np.array(seq_09_ates)
seq_09_res = np.array(seq_09_res)
print("\n   Trajectory error: {:0.3f}, std: {:0.3f}\n".format(np.mean(seq_09_ates), np.std(seq_09_ates)))
print("\n   Rotation error: {:0.3f}, std: {:0.3f}\n".format(np.mean(seq_09_res), np.std(seq_09_res)))

# save_path = os.path.join(opt.load_weights_folder, "poses.npy")
# np.save(save_path, pred_poses)
# print("-> Predictions saved to", save_path)

100%|██████████| 1590/1590 [04:49<00:00,  5.49it/s]


   Trajectory error: 0.011, std: 0.010


   Rotation error: 0.001, std: 0.000



In [3]:
seq_10_ates = []
seq_10_res = []
for i, data in enumerate(tqdm(dataloader_10)):
    with torch.no_grad():
        # Get input and target tensor keys
        input_keys = list(filter(lambda x: "input" in x, data.keys()))
        target_keys = list(filter(lambda x: "target" in x, data.keys()))
        tensor_keys = input_keys + target_keys
        
        # Possibly transfer to Cuda
        for k, v in data.items():
            if k in tensor_keys:
                data[k] = v.cuda(non_blocking=True)
                
        aug_data = aug(data)
        out = model(aug_data)
        pose_t = out['pose_b'][0][:, :3, 3].double().cpu()
        gt_t = data['target_pose'][:, :3, 3].double().cpu()
        
        pose_R = out['pose_b'][0][:, :3, :3].double().cpu()[0]
        gt_R = data['target_pose'][:, :3, :3].double().cpu()[0]
        
        R = torch.matmul(gt_R, torch.inverse(pose_R))
        s = np.linalg.norm([R[0, 1]-R[1, 0],
                            R[1, 2]-R[2, 1],
                            R[0, 2]-R[2, 0]])
        c = np.trace(R) - 1
        RE = np.arctan2(s, c)
        
        ate = compute_ate(gt_t, pose_t)
        seq_10_ates.append(np.array(ate))
        seq_10_res.append(RE)
        
seq_10_ates = np.array(seq_10_ates)
seq_10_res = np.array(seq_10_res)
print("\n   Trajectory error: {:0.3f}, std: {:0.3f}\n".format(np.mean(seq_10_ates), np.std(seq_10_ates)))
print("\n   Rotation error: {:0.3f}, std: {:0.3f}\n".format(np.mean(seq_10_res), np.std(seq_10_res)))

# save_path = os.path.join(opt.load_weights_folder, "poses.npy")
# np.save(save_path, pred_poses)
# print("-> Predictions saved to", save_path)

100%|██████████| 1200/1200 [03:55<00:00,  5.10it/s]


   Trajectory error: 0.009, std: 0.009


   Rotation error: 0.001, std: 0.001



In [ ]:
def compute_pose_errors(gt, pred):
    RE = 0
    for (current_gt, current_pred) in zip(gt, pred):
        snippet_length = current_gt.shape[0]
        scale_factor = torch.sum(current_gt[..., -1] * current_pred[..., -1]) / torch.sum(current_pred[..., -1] ** 2)
        ATE = torch.norm((current_gt[..., -1] - scale_factor * current_pred[..., -1]).reshape(-1)).cpu().numpy()
        R = current_gt[..., :3] @ current_pred[..., :3].transpose(-2, -1)
        for gt_pose, pred_pose in zip(current_gt, current_pred):
            # Residual matrix to which we compute angle's sin and cos
            R = (gt_pose[:, :3] @ torch.inverse(pred_pose[:, :3])).cpu().numpy()
            s = np.linalg.norm([R[0, 1]-R[1, 0],
                                R[1, 2]-R[2, 1],
                                R[0, 2]-R[2, 0]])
            c = np.trace(R) - 1
            # Note: we actually compute double of cos and sin, but arctan2 is invariant to scale
            RE += np.arctan2(s, c)

    return [ATE/snippet_length, RE/snippet_length]